In [1]:
import yfinance as yf
import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier

In [2]:
if os.path.exists("bitcoin_trades.csv"):
    bitcoinData = pd.read_csv("bitcoin_trades.csv", index_col=0)
    bitcoinData.index = pd.to_datetime(bitcoinData.index, unit='s')

# bitcoinData = bitcoinData.dropna()

In [3]:
# bitcoinData.plot.line(y="Close", use_index=True)

In [4]:
bitcoinData["next_minute"] = bitcoinData["Close"].shift(-1)
bitcoinData["target"] = (bitcoinData["next_minute"] > bitcoinData["Close"]).astype(int)

In [5]:
bitcoinData = bitcoinData.loc["2018-01-01":]
bitcoinData = bitcoinData.fillna(method="ffill")

In [6]:
horizons = [5,30,90,120]
new_predictors = []

for horizon in horizons:
    rolling_avg = bitcoinData.rolling(horizon).mean().fillna(method="ffill")
    # print(rolling_avg)
    
    ratio_column = f"Close_Ratio_{horizon}"
    bitcoinData[ratio_column] = bitcoinData["Close"] / rolling_avg["Close"]
    
    trend_column = f"Trend_{horizon}"
    bitcoinData[trend_column] = bitcoinData.shift(1).rolling(horizon).sum()["target"]
    
    new_predictors+= [ratio_column, trend_column]
    

In [7]:
bitcoinData = bitcoinData.iloc[120:]
bitcoinData

,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,next_minute,target,Close_Ratio_5,Trend_5,Close_Ratio_30,Trend_30,Close_Ratio_90,Trend_90,Close_Ratio_120,Trend_120
Timestamp,,,,,,,,,,,,,,,,,
2018-01-01 02:00:00,13355.00,13380.86,13355.00,13380.00,1.486381,19864.561129,13364.382575,13441.78,1,1.002172,3.0,0.992320,11.0,0.983648,38.0,0.981641,49.0
2018-01-01 02:01:00,13380.82,13441.78,13380.81,13441.78,7.707694,103290.214000,13400.923217,13452.34,1,1.005372,4.0,0.997410,12.0,0.988397,39.0,0.986444,49.0
2018-01-01 02:02:00,13420.54,13457.00,13390.02,13452.34,9.439213,126866.537570,13440.372310,13420.01,0,1.004354,5.0,0.998630,13.0,0.989391,39.0,0.987458,50.0
2018-01-01 02:03:00,13450.80,13501.44,13403.10,13420.01,6.786561,91377.193288,13464.433014,13490.94,1,1.000759,4.0,0.996666,13.0,0.987248,39.0,0.985328,50.0
2018-01-01 02:04:00,13420.01,13497.25,13391.04,13490.94,1.931469,25997.060447,13459.737692,13532.18,1,1.004013,4.0,1.002115,14.0,0.992608,40.0,0.990725,51.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-30 23:56:00,58714.31,58714.31,58686.00,58686.00,1.384487,81259.372187,58692.753339,58685.81,0,0.999630,1.0,1.000154,12.0,0.999865,40.0,0.999263,54.0
2021-03-30 23:57:00,58683.97,58693.43,58683.97,58685.81,7.294848,428158.146640,58693.226508,58723.84,1,0.999675,1.0,1.000176,11.0,0.999891,40.0,0.999270,54.0
2021-03-30 23:58:00,58693.43,58723.84,58693.43,58723.84,1.705682,100117.070370,58696.198496,58760.59,1,1.000236,2.0,1.000823,12.0,1.000560,41.0,0.999923,54.0


In [8]:
model = RandomForestClassifier(n_estimators=100, min_samples_split=100, random_state=1)

train = []

for i in range(0, len(bitcoinData), 100):
    subset = bitcoinData.iloc[i:i+100]
    train.append(subset)
    
test = bitcoinData.iloc[-100:]

predictors = ["Close", "Volume_(BTC)", "Open", "High", "Low"]
model.fit(train[predictors], train["target"])

TypeError: list indices must be integers or slices, not list